# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [ ]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
dataset_path = '/content/drive/MyDrive/DMEyF/2024/datos/'
dataset_file = 'competencia_01.csv'

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [ ]:
%%sql
select
    Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
from competencia_01 limit 10

,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor
0,-757,-1122,-757,-1122
1,-729,-1094,-729,-1094
2,-698,-1063,-698,-1063
3,-668,-1033,-668,-1033
4,-637,-1002,-637,-1002
5,-607,-972,-607,-972
6,-545,-637,-545,-637
7,-517,-609,-517,-609
8,-486,-578,-486,-578
9,-456,-548,-456,-548


Lo siguiente es querer operar dos variables, como por ejemplo sumarla. Esto es sencillo


In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,0.00,48086.18,48086.18
1,0.00,47799.46,47799.46
2,0.00,50765.09,50765.09
3,0.00,56285.43,56285.43
4,187297.56,53633.54,240931.10
5,0.00,48645.84,48645.84
6,0.00,5775.84,5775.84
7,0.00,4781.11,4781.11
8,0.00,6814.85,6814.85
9,0.00,6898.83,6898.83


Pero un DS de a de veras mirará los datos y se encontrará con un campo que es null cuando se lo suma a otro dará null.

In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,NaN,5022.58,NaN
1,NaN,7823.99,NaN
2,NaN,8263.66,NaN
3,NaN,3678.81,NaN
4,NaN,8112.36,NaN
5,NaN,12741.87,NaN
6,NaN,18583.44,NaN
7,NaN,19414.00,NaN
8,NaN,3696.07,NaN
9,NaN,20106.40,NaN


Esto no siempre es deseable y puede ser fácilmente evitable

In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,0.00,48086.18,48086.18
1,0.00,47799.46,47799.46
2,0.00,50765.09,50765.09
3,0.00,56285.43,56285.43
4,187297.56,53633.54,240931.10
5,0.00,48645.84,48645.84
6,0.00,5775.84,5775.84
7,0.00,4781.11,4781.11
8,0.00,6814.85,6814.85
9,0.00,6898.83,6898.83


In [ ]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);


,Success


In [ ]:
%%sql
select distinct
    Master_msaldototal
    , Visa_msaldototal
    , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10


,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,NaN,52897.58,52897.58
1,NaN,25788.71,25788.71
2,NaN,671.66,671.66
3,NaN,17.04,17.04
4,NaN,7694.94,7694.94
5,NaN,12501.48,12501.48
6,NaN,21443.85,21443.85
7,NaN,23562.11,23562.11
8,NaN,24483.42,24483.42
9,NaN,22239.63,22239.63


TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [ ]:
%%sql
select
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01
order by numero_de_cliente, cliente_antiguedad


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,foto_mes,numero_de_cliente,cliente_antiguedad,cliente_antiguedad_2,cliente_antiguedad_3,cliente_antiguedad_4,cliente_antiguedad_5,cliente_antiguedad_6
0,202101,249221109,300,1,0.947564,0.947947,4,10
1,202102,249221109,301,2,0.947831,0.948213,4,10
2,202103,249221109,302,3,0.948291,0.948670,4,10
3,202104,249221109,303,4,0.948638,0.949016,4,10
4,202105,249221109,304,5,0.948950,0.949327,4,10
...,...,...,...,...,...,...,...,...
981941,202106,1598444941,1,1,0.000000,0.001553,1,1
981942,202106,1598471047,1,1,0.000000,0.001553,1,1
981943,202106,1598515897,1,1,0.000000,0.001553,1,1
981944,202106,1598517059,1,1,0.000000,0.001553,1,1


Qué paso? use las hermosas funciones analíticas de SQL. Al campo cliente_antiguedad (que no sufre de data drifting, solo esta para dar el ejemplo) para cada período (partition by foto_mes) la ordeno (order by cliente_antiguedad) y luego calculo las métricas de orden que pueden encontrar acá https://duckdb.org/docs/sql/window_functions.html#general-purpose-window-functions.

Seguiremos usando las funciones analíticas de SQL, esta vez para calcular features que utilizan valores del pasado.

Qué pasa si quiero agregar un feature que muestre el valor del periodo anterior?


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,249223005,202101,182,NaN
1,249223005,202102,208,182.0
2,249223005,202103,201,208.0
3,249223005,202104,194,201.0
4,249223005,202105,171,194.0
5,249223005,202106,172,171.0
6,249237079,202101,141,NaN
7,249237079,202102,149,141.0
8,249237079,202103,153,149.0
9,249237079,202104,160,153.0


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,249223005,202101,182,NaN,NaN,NaN
1,249223005,202102,208,182.0,26.0,NaN
2,249223005,202103,201,208.0,-7.0,19.0
3,249223005,202104,194,201.0,-7.0,-14.0
4,249223005,202105,171,194.0,-23.0,-30.0
5,249223005,202106,172,171.0,1.0,-22.0
6,249237079,202101,141,NaN,NaN,NaN
7,249237079,202102,149,141.0,8.0,NaN
8,249237079,202103,153,149.0,4.0,12.0
9,249237079,202104,160,153.0,7.0,11.0


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,lag_2_ctrx_quarter,lag_3_ctrx_quarter,avg_3_ctrx_quarter
0,249221109,202106,199,199.0,188.0,174.0,190.000000
1,249221109,202105,199,188.0,174.0,161.0,180.500000
2,249221109,202104,188,174.0,161.0,166.0,172.250000
3,249221109,202103,174,161.0,166.0,NaN,167.000000
4,249221109,202102,161,166.0,NaN,NaN,163.500000
5,249221109,202101,166,NaN,NaN,NaN,166.000000
6,249221468,202106,191,182.0,182.0,170.0,181.250000
7,249221468,202105,182,182.0,170.0,154.0,172.000000
8,249221468,202104,182,170.0,154.0,145.0,162.750000
9,249221468,202103,170,154.0,145.0,NaN,156.333333


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_3,ctrx_quarter_max_3,ctrx_quarter_min_3
0,249223005,202101,182,182.000000,182,182
1,249223005,202102,208,195.000000,208,182
2,249223005,202103,201,197.000000,208,182
3,249223005,202104,194,196.250000,208,182
4,249223005,202105,171,193.500000,208,171
5,249223005,202106,172,184.500000,201,171
6,249237079,202101,141,141.000000,141,141
7,249237079,202102,149,145.000000,149,141
8,249237079,202103,153,147.666667,153,141
9,249237079,202104,160,150.750000,160,141


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


... Alguno dirá "tenemos que escribir todo esto a mano? Son muchas variables!". Bueno no, use los conocimientos de programación para que la computadora trabaje para usted. Si tenemos una lista de campos


In [ ]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']


Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables


In [ ]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as ctrx_{campo}_slope_3"
print(nuevos_features)



, regr_slope(active_quarter, cliente_antiguedad) over ventana_3 as ctrx_active_quarter_slope_3
, regr_slope(cliente_vip, cliente_antiguedad) over ventana_3 as ctrx_cliente_vip_slope_3
, regr_slope(internet, cliente_antiguedad) over ventana_3 as ctrx_internet_slope_3
, regr_slope(cliente_edad, cliente_antiguedad) over ventana_3 as ctrx_cliente_edad_slope_3
, regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_3 as ctrx_cliente_antiguedad_slope_3
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as ctrx_mrentabilidad_slope_3





Con la salida de esa celda, arme la query agregando las nuevas líneas y la ejecuta.

Lo que acabamos de hacer de manera muy simple es como "funcionan" sistemas como **dbt** que están tan de moda en el mundo de los datos.

La última reflexión, la creación de nuevas features es un proceso computacionalmente rápido pero intenso. Si ejecutó lo anterior pudo haber visto que en poco minutos tenía sus nuevas variables. Pero, también pudo haberle fallado por temas de recursos. Miles de variables necesitan los recursos adecuados. Use la nube, una máquina grande, al menos que sepa bien como optimizar las queries.


Y a no olvidarse guardar las nueva tabla

In [ ]:
%%sql
COPY competencia_01 TO '{dataset_path}competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE);
